In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [2]:
# Defining Functions

def click_to_pk_page_redux():
    list_more_links = driver.find_elements(By.PARTIAL_LINK_TEXT,'more')
    page_exists = False
    for link in list_more_links:
        if len(link.text.split(' ')) == 2:
            link.click() # Temporary, need to identify by new thing
            page_exists = True
            return page_exists
    return page_exists
    
def find_general_info(soup):
    #title = soup.find('h1',"TitleHeader__TitleText-sc-1wu6n3d-0 dxSWFG").string
    title = soup.find('h1')
    title =['Title',title.text]
    
    #year_of_release = ''
    #try: 
    #    year_of_release = soup.find('a','ipc-link ipc-link--baseAlt ipc-link--inherit-color TitleBlockMetaData__StyledTextLink-sc-12ein40-1 rgaOW').string
    #except:
    #    year_of_release = soup.find('a',"ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
    #year_of_release = ['Year',year_of_release]
    
    imdb_rating = ''
    try:
        imdb_rating = soup.find('span',"AggregateRatingButton__RatingScore-sc-1ll29m0-1 iTLWoV").string
        imdb_rating = ['IMDB_Rating',imdb_rating]
    except:
        imdb_rating = ['IMDB_Rating',imdb_rating]
    
    description = ''
    try:
        description = soup.find('span',"GenresAndPlot__TextContainerBreakpointL-cum89p-1 gwuUFD").string
        description = ['Movie_Description',description]
    except:
        try: 
            description = soup.find(class_ = 'GenresAndPlot__TextContainerBreakpointXL-cum89p-2 gCtawA')
            description = ['Movie_Description',description]
        except:
            description = ['Movie_Description',description]
    
    release_date = ''
    try:
        temp = soup.find_all('ul',class_='ipc-metadata-list ipc-metadata-list--dividers-all ipc-metadata-list--base')
        release_date = temp[1].find(class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link').string
        release_date = release_date.split('(')[0]
        release_date = ['Date_of_Release',release_date]
    except:
        try:
            temp = driver.find_element(By.PARTIAL_LINK_TEXT,'(United States)')
            release_date = temp.text.split('(')[0]
            release_date = ['Date_of_Release',release_date]
        except:
            release_date = ['Date_of_Release',release_date]
    
    return [title,imdb_rating,description,release_date]

def find_plot_keywords():
    if click_to_pk_page_redux() == True:
        soup = bs(driver.page_source,'html.parser')
        plot_keywords = []
        rows = soup.find('tbody').find_all('tr')
        for row in rows:
            two_cols = row.find_all(class_='soda sodavote')
            for col in two_cols:
                sub_col = col.find(class_='interesting-count-text').a.string[1:-1]
                val = col['data-item-keyword']
                plot_keywords.append([val,sub_col])
        top_ten = plot_keywords[0:10]
        top_ten_list = []
        for item in top_ten:
            top_ten_list.append(item[0])
        return ['Plot_Keywords',top_ten_list]
    else:
        return ['Plot_Keywords',['none']]
    
def click_next():
    htmlelement= driver.find_element(By.TAG_NAME,'html')
    htmlelement.send_keys(Keys.HOME)
    desc = driver.find_element(By.CLASS_NAME,'desc')
    next_link = desc.find_element(By.PARTIAL_LINK_TEXT,'Next')
    next_link.click()
    return

def get_page_count():
    desc = driver.find_element(By.CLASS_NAME,'desc').text
    if '-' in desc:
        split = desc.split(' ')
        split_split = split[0].split('-')
        for i in range(len(split_split)):
            split_split[i] = split_split[i].replace(',','')
        page_range = int(split_split[-1])-int(split_split[0])+1
        return page_range
    elif 'to' in desc:
        split = desc.split(' ')
        for i in range(len(split)):
            split[i] = split[i].replace(',','')
        page_range = int(split[2])-int(split[0])+1
    else:
        page_range = int(desc.split(' ')[0])
    return page_range
    
def get_total_count():
    desc = driver.find_element(By.CLASS_NAME,'desc').text
    split = desc.split(' ')
    if 'of' in split:
        number = split[split.index('of')+1]    
    else:
        number = split[0]
    number = number.replace(',','')
    #if ',' in number:
    #    number = int(number.replace(',',''))
    #else:
    #    number = int(number)
    return int(number)
### SOMETIMES IT's TO AND SOMETIMES IT's -.... figure out better way to get the numbers out

def get_movie_info_modified():
    soup = bs(driver.page_source,'html.parser')
    movie_data = find_general_info(soup)
    boxoffice = find_boxoffice_modified(soup)
    
    for item in boxoffice:
        movie_data.append(item)
    
    values = []
    for item in movie_data:
        values.append(item)    
    return(values)

def click_to_movie_page_modified(i):
    area = driver.find_element(By.CLASS_NAME, 'lister-list')
    rows = area.find_elements(By.CLASS_NAME, 'lister-item-header')
    link_name = rows[i].text.split(' ')[1]
    rows[i].find_element(By.PARTIAL_LINK_TEXT,link_name).click()
    return

def get_keyword_list():  
    keyword_list = []
    done = False
    while done == False:
        keyword = input('Please enter the plot keyword you would like movie data for.\n\
If searching by multiple keywords at once, separate them with forward-slashes. \nEnter: ')
        split = keyword.split('/')
        for i in range(len(split)):
            if split[i][0] == ' ':
                split[i] = split[i][1:]
        for i in range(len(split)):
            split[i] = split[i].replace(' ','-')
        keyword_list.append(split)
        search = input('Would you like data for another keyword? \nResults are saved\
in separate .csv files for each keyword you input. \nEnter y/n: ')
        if search == 'n':
            done = True
    return keyword_list

def get_keyword_url(keyword):
    if len(keyword) == 1:
        keyword_url = f'https://www.imdb.com/search/keyword/?keywords={keyword[0]}&ref_=kw_ref_typ&sort=moviemeter,asc&mode=detail&page=1&title_type=movie'
        return keyword_url
    elif len(keyword) > 1:
        # alien%2Couter-space%2Cspaceship
        keywords = '%2C'.join(keyword)
        keyword_url = f'https://www.imdb.com/search/keyword/?keywords={keywords}&ref_=kw_ref_key&mode=detail&page=1&title_type=movie&sort=moviemeter,asc'
        return keyword_url
    
def find_boxoffice_modified(soup):
    items = soup.find_all('li','ipc-metadata-list__item BoxOffice__MetaDataListItemBoxOffice-sc-40s2pl-2 gwNUHl')
    possible_items = ['Budget','Gross US & Canada','Opening weekend US & Canada','Gross worldwide']
    values = [['Budget',""],['Gross US & Canada',""],['Opening weekend US & Canada',""],['Gross worldwide',""]]
    for item in items:
        amount = ''
        label = item.find(class_='ipc-metadata-list-item__label').string
        temp = item.find(class_='ipc-metadata-list-item__list-content-item')
        values[possible_items.index(label)][-1] = temp.text
    final_list = []
    for i in range(4):
        final_list.append(values[i])
    return final_list

In [ ]:
val_list = []
keyword_list = get_keyword_list()
driver = webdriver.Chrome()
for keyword in keyword_list: 
    url = get_keyword_url(keyword)
    driver.get(url)
    titles_count = get_total_count()
    if titles_count > 200:
        titles_count = 150
    for j in range(titles_count // 50 + 1):
        for i in range(get_page_count()):
            try:
                click_to_movie_page_modified(i)
            except:
                continue
            movie_info = get_movie_info_modified()
            print(f'got movie info for {movie_info[0][-1]}')
            values = []
            for item in movie_info:
                values.append(item[-1])
            val_list.append(values)
            driver.back()        
        if j == titles_count // 50:
            continue
        else:
            time.sleep(5)
            click_next()
            time.sleep(5)
    # Wasn't remembering columns variable after the try block. Probably a good thing.
    columns = ['Title','IMDB_Rating','Movie_Description','Release_Date', 'Budget',\
              'Gross_US_Canada', 'Opening_Weekend_US_Canada', 'Gross_Worldwide']
    for i in range(len(keyword)):
        keyword[i] = keyword[i].replace('-','_')
    df = pd.DataFrame(val_list)
    df.columns = columns
    df.to_csv(f'keyword_{"_".join(keyword)}.csv')
    print(f'\n\n CSV SUCCESSFULLY CREATED: keyword_{"_".join(keyword)}.csv')
print(f'Grabbed {len(df)} out of {titles_count} samples')
driver.quit()

Please enter the plot keyword you would like movie data for.
If searching by multiple keywords at once, separate them with forward-slashes. 
Enter: swamp
Would you like data for another keyword? 
Results are savedin separate .csv files for each keyword you input. 
Enter y/n: n
got movie info for The Lord of the Rings: The Fellowship of the Ring
got movie info for Avengers: Infinity War
got movie info for The Princess Bride
got movie info for The Lord of the Rings: The Two Towers
got movie info for The Departed
got movie info for Chaos Walking
got movie info for Star Wars: Episode V - The Empire Strikes Back
